In [21]:
# This model is highly based on the MNIST 
# tutorial provided by TensorFlow with MNIST dataset 
# for Kaggle Competition

import numpy as np
import pandas as pd
from sklearn import cross_validation
from sklearn import datasets

In [24]:
import tensorflow as tf
sess = tf.InteractiveSession()

In [2]:
# getData is used to get train and test sets 
# CorssValidation set is to be added
data=pd.read_csv('../dataset/train.csv', header=0, dtype=np.float64)

In [3]:
# Check the shape of data
data.shape

(42000, 785)

In [4]:
# See what we have in the data
data.head(3)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
# Get the features 'pixel0, ..., pixel783' as X, 'label' as y.
X_data=data.iloc[0::,1::].values
y_data=data['label'].values

In [6]:
# K_fold cross_validation.
# For this ipython notebook, we actually just work in the last fold split.
# For later *.py, we can include the training in this for loop.
k_fold=cross_validation.KFold(n=len(y_data), n_folds=7)
for train, test in k_fold:
    train_x=X_data[train]
    test_x=X_data[test]
    train_y=y_data[train]
    test_y=y_data[test]

In [7]:
# Make label of 1, 2, 3, 4.... into a hot vector 
# whose ith index is the label and the rest are 0
def make_hot(lables):
    result = []
    for i in range(len(lables)):
        temp = [0.0] * 10
        temp[int(lables[i])] = 1.0
        result.append(temp)
    return result

In [8]:
# Transform vector of the form [1, 3, 4...] into 
# hot vector [1, 0, 0, 0 ...] [0, 0, 1, 0, ...]....
train_y = make_hot(train_y)
test_y = make_hot(test_y)

In [9]:
# Just check for fun.
train_y[0:10:]

[[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]

In [10]:
def normalize(myList):
    for i in range(len(myList)):
        myList[i] = [k/255 for k in myList[i]] 
    return myList

In [11]:
train_x = normalize(train_x)
test_x = normalize(test_x)

In [18]:
# A softmax regression containing only one layer of 
# Network. The regression is in form of y = softmax(Wx + b)
# W is the weigt and b is the bias 

# First we need a placeholder for each input value x
# Target output y_
# None corresponds to the batch size 
x = tf.placeholder("float", shape=[None, 784])
y_ = tf.placeholder("float", shape=[None, 10])

# Then we decalre W and b as Variables for model parameters
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

In [26]:
# Before Variables can be used in a session 
# They must be initialized
sess.run(tf.initialize_all_variables())

In [30]:
# Weight Initialization 
# symmetry beaking 
def weight_variable(shape):
    inital = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(inital)

# avoid dead neurons
def bias_variable(shape):
    inital = tf.constant(0.1, shape=shape)
    return tf.Variable(inital)

In [28]:
# Convolution and pooling
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [32]:
# First convolutional layer 
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

x_image = tf.reshape(x, [-1,28,28,1])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [33]:
# Second layer 
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [35]:
# Densely connected layer
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [36]:
# Dropout 
keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [37]:
# Readout layer 
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

In [52]:
# Similar to the softMax network 
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

batchLoop = len(train_x) / 50
sess.run(tf.initialize_all_variables())
for i in range(batchLoop):
    batch_xs, batch_ys = train_x[i*(50):(i+1)*50], train_y[i*(50):(i+1)*50]
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={x: batch_xs, y_: batch_ys, keep_prob: 1.0})
        print("Step %d, training accuracy %g"%(i, train_accuracy))
    train_step.run(feed_dict={x: batch_xs, y_:batch_ys, keep_prob: 0.5})

Step 0, training accuracy 0.16
Step 100, training accuracy 0.8
Step 200, training accuracy 0.9
Step 300, training accuracy 0.94
Step 400, training accuracy 1
Step 500, training accuracy 0.94
Step 600, training accuracy 0.94
Step 700, training accuracy 0.96


In [53]:
print("test accuracy %g"%accuracy.eval(feed_dict={
    x: test_x, y_: test_y, keep_prob: 1.0}))

test accuracy 0.961167
